<a href="https://colab.research.google.com/github/gummadhav/Let_us_Learn/blob/main/GPU_WarmUp_Trick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
%%shell
# Update package lists
apt-get update

# Install essential build tools (if not already present, good practice)
apt-get install -y build-essential

# Install OpenCV development libraries (C++ headers and shared libraries)
# 'libopencv-dev' provides the development files
# 'python3-opencv' is for Python, but sometimes pulled in as a dependency
apt-get install -y libopencv-dev python3-opencv

# Verify OpenCV version (optional, but good for checking)
pkg-config --modversion opencv4 # For OpenCV 4.x
# Or for older versions: pkg-config --modversion opencv

/bin/bash: line 1: fg: no job control
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,933 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,521 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,209 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:13 http://

In [2]:
!pip install nvcc4jupyter

In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpuj92vcmu".


In [4]:
%%shell
nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [5]:
%%shell
nvidia-smi

Fri Aug 15 14:51:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [14]:
%%writefile cpu_vs_gpu.cu

#include <opencv2/opencv.hpp>
#include <vector>
#include <string>
#include <filesystem>
#include <iostream>
#include "cuda_runtime.h"

std::vector<std::string> get_image_paths(const std::string& folder) {
    std::vector<std::string> paths;
    for (const auto& entry : std::filesystem::directory_iterator(folder)) {
        if (entry.path().extension() == ".bmp") {
            paths.push_back(entry.path().string());
        }
    }
    return paths;
}

// ----- Gaussian Filter Kernel (CUDA) -----
__global__
void gaussian_filter_kernel(const uchar* input, uchar* output, int width, int height, const float* kernel) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    float sum = 0, kSum = 0;
    for (int dy = -1; dy <= 1; ++dy) {
        for (int dx = -1; dx <= 1; ++dx) {
            int ix = min(max(x+dx,0), width-1);
            int iy = min(max(y+dy,0), height-1);
            float k = kernel[(dy+1)*3 + (dx+1)];
            sum += input[iy*width + ix] * k;
            kSum += k;
        }
    }
    float result = kSum > 0 ? sum / kSum : 0;
    // Clamp and convert
    result = min(max(result, 0.f), 255.f);
    output[y*width + x] = static_cast<uchar>(result);
}

// ----- Laplacian Edge Detection Kernel (CUDA) -----
__global__
void laplacian_filter_kernel(const uchar* input, uchar* output, int width, int height) {
    int kernel[3][3] = { {-1,-1,-1}, {-1,8,-1}, {-1,-1,-1} };
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    int sum = 0;
    for (int dy = -1; dy <= 1; ++dy) {
        for (int dx = -1; dx <= 1; ++dx) {
            int ix = min(max(x+dx,0), width-1);
            int iy = min(max(y+dy,0), height-1);
            sum += input[iy*width + ix] * kernel[dy+1][dx+1];
        }
    }
    // Offset and clamp
    int result = sum + 128;
    result = min(max(result, 0), 255);
    output[y*width + x] = static_cast<uchar>(result);
}

#define CHECK_CUDA(call) \
    { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            std::cerr << "CUDA error: " << cudaGetErrorString(err) << " at " << __FILE__ << ":" << __LINE__ << std::endl; \
            exit(1); \
        } \
    }

__global__ void empty_kernel(int* p) { *p = 1; }
// Add this before your 'for' loop
void warm_up() {
    // Launch a simple, small kernel to prime the CUDA context
    int* d_temp;
    CHECK_CUDA(cudaMalloc(&d_temp, sizeof(int)));
    int temp = 0;
    CHECK_CUDA(cudaMemcpy(d_temp, &temp, sizeof(int), cudaMemcpyHostToDevice));
    dim3 block(1, 1), grid(1, 1);

    empty_kernel<<<grid, block>>>(d_temp);
    CHECK_CUDA(cudaDeviceSynchronize());
    CHECK_CUDA(cudaFree(d_temp));
}

int main() {
    std::string input_folder = "input_images";
    std::string output_folder = "output_images";
    auto paths = get_image_paths(input_folder);

    // Perform a warm-up run to reduce overhead on the first timed iteration
    if (!paths.empty()) {
        std::cout << "Warming up CUDA..." << std::endl;
        warm_up();
        std::cout << "Warm-up complete." << std::endl;
    }

    float gaussKernel[9] = {1.f/16, 2.f/16, 1.f/16, 2.f/16, 4.f/16, 2.f/16, 1.f/16, 2.f/16, 1.f/16};

    for (const auto& path : paths) {
        std::cout << "Processing: " << path << std::endl;
        cv::TickMeter total_cpu_timer;
        total_cpu_timer.start();

        cv::Mat img = cv::imread(path, cv::IMREAD_GRAYSCALE);
        if (img.empty() || img.type() != CV_8U || img.channels() != 1) {
            std::cerr << "Invalid image: " << path << std::endl;
            continue;
        }
        int width = img.cols, height = img.rows;
        size_t size = width * height * sizeof(uchar);

        // ----------- CPU Processing -----------
        cv::TickMeter timer_cpu_gauss, timer_cpu_laplace;
        cv::Mat cpu_gauss, cpu_laplace;
        timer_cpu_gauss.start();
        cv::GaussianBlur(img, cpu_gauss, cv::Size(3, 3), 1, 1, cv::BORDER_DEFAULT);
        timer_cpu_gauss.stop();

        timer_cpu_laplace.start();
        cv::Laplacian(cpu_gauss, cpu_laplace, CV_8U, 3, 1, 128); // offset=128
        timer_cpu_laplace.stop();

        std::string cpu_output = output_folder + "/" + std::filesystem::path(path).stem().string() + "_cpu.bmp";
        if (!cv::imwrite(cpu_output, cpu_laplace)) {
            std::cerr << "Failed to save CPU image: " << cpu_output << std::endl;
        }
        total_cpu_timer.stop();

        // ----------- GPU Processing -----------
        cv::TickMeter timer_gpu_total, timer_gpu_gauss, timer_gpu_laplace;
        timer_gpu_total.start();
        uchar *d_input, *d_gauss, *d_laplace;
        float* d_gkernel;
        CHECK_CUDA(cudaMallocManaged(&d_input, size));
        CHECK_CUDA(cudaMallocManaged(&d_gauss, size));
        CHECK_CUDA(cudaMallocManaged(&d_laplace, size));
        CHECK_CUDA(cudaMallocManaged(&d_gkernel, 9 * sizeof(float)));

        CHECK_CUDA(cudaMemcpy(d_input, img.data, size, cudaMemcpyHostToDevice));
        CHECK_CUDA(cudaMemcpy(d_gkernel, gaussKernel, 9 * sizeof(float), cudaMemcpyHostToDevice));

        dim3 block(16,16), grid((width+15)/16, (height+15)/16);

        timer_gpu_gauss.start();
        gaussian_filter_kernel<<<grid, block>>>(d_input, d_gauss, width, height, d_gkernel);
        timer_gpu_gauss.stop();

        CHECK_CUDA(cudaDeviceSynchronize());
        CHECK_CUDA(cudaGetLastError());

        timer_gpu_laplace.start();
        laplacian_filter_kernel<<<grid, block>>>(d_gauss, d_laplace, width, height);
        timer_gpu_laplace.stop();
        CHECK_CUDA(cudaDeviceSynchronize());
        CHECK_CUDA(cudaGetLastError());

        cv::Mat out(height, width, CV_8U);
        CHECK_CUDA(cudaMemcpy(out.data, d_laplace, size, cudaMemcpyDeviceToHost));

        std::string gpu_output = output_folder + "/" + std::filesystem::path(path).stem().string() + "_gpu.bmp";
        if (!cv::imwrite(gpu_output, out)) {
            std::cerr << "Failed to save GPU image: " << gpu_output << std::endl;
        }

        CHECK_CUDA(cudaFree(d_input));
        CHECK_CUDA(cudaFree(d_laplace));
        CHECK_CUDA(cudaFree(d_gkernel));
        timer_gpu_total.stop();

        //total_timer.stop();
        // ------- Print stage timings --------
        std::cout << "CPU GaussianBlur:    " << timer_cpu_gauss.getTimeMilli()    << " ms" << std::endl;
        std::cout << "CPU Laplacian:       " << timer_cpu_laplace.getTimeMilli()  << " ms" << std::endl;
        std::cout << "Total image time:    " << total_cpu_timer.getTimeMilli()    << " ms" << std::endl << std::endl;

        std::cout << "GPU Gaussian kernel: " << timer_gpu_gauss.getTimeMilli()    << " ms" << std::endl;
        std::cout << "GPU Laplace kernel: " << timer_gpu_laplace.getTimeMilli() << " ms" << std::endl;
        std::cout << "GPU Total:           " << timer_gpu_total.getTimeMilli()    << " ms" << std::endl;
        std::cout << "\n";
    }
    return 0;
}


Overwriting cpu_vs_gpu.cu


In [15]:
%%shell
nvcc cpu_vs_gpu.cu -o cpu_vs_gpu $(pkg-config --cflags --libs opencv4) -arch=sm_75

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [16]:
%%shell
ls -l cpu_vs_gpu

-rwxr-xr-x 1 root root 1079864 Aug 15 14:57 cpu_vs_gpu


In [17]:
%%shell
./cpu_vs_gpu

Warming up CUDA...
Warm-up complete.
Processing: input_images/greenland_grid_velo.bmp
CPU GaussianBlur:    2.22504 ms
CPU Laplacian:       1.4323 ms
Total image time:    9.58302 ms

GPU Gaussian kernel: 0.036262 ms
GPU Laplace kernel: 0.012797 ms
GPU Total:           3.4869 ms

Processing: input_images/lena.bmp
CPU GaussianBlur:    0.157529 ms
CPU Laplacian:       0.271312 ms
Total image time:    1.2947 ms

GPU Gaussian kernel: 0.007794 ms
GPU Laplace kernel: 0.003726 ms
GPU Total:           0.797124 ms



In [10]:
%%writefile cpu_vs_gpu_no_warmup.cu

#include <opencv2/opencv.hpp>
#include <vector>
#include <string>
#include <filesystem>
#include <iostream>
#include "cuda_runtime.h"

std::vector<std::string> get_image_paths(const std::string& folder) {
    std::vector<std::string> paths;
    for (const auto& entry : std::filesystem::directory_iterator(folder)) {
        if (entry.path().extension() == ".bmp") {
            paths.push_back(entry.path().string());
        }
    }
    return paths;
}

// ----- Gaussian Filter Kernel (CUDA) -----
__global__
void gaussian_filter_kernel(const uchar* input, uchar* output, int width, int height, const float* kernel) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    float sum = 0, kSum = 0;
    for (int dy = -1; dy <= 1; ++dy) {
        for (int dx = -1; dx <= 1; ++dx) {
            int ix = min(max(x+dx,0), width-1);
            int iy = min(max(y+dy,0), height-1);
            float k = kernel[(dy+1)*3 + (dx+1)];
            sum += input[iy*width + ix] * k;
            kSum += k;
        }
    }
    float result = kSum > 0 ? sum / kSum : 0;
    // Clamp and convert
    result = min(max(result, 0.f), 255.f);
    output[y*width + x] = static_cast<uchar>(result);
}

// ----- Laplacian Edge Detection Kernel (CUDA) -----
__global__
void laplacian_filter_kernel(const uchar* input, uchar* output, int width, int height) {
    int kernel[3][3] = { {-1,-1,-1}, {-1,8,-1}, {-1,-1,-1} };
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= width || y >= height) return;
    int sum = 0;
    for (int dy = -1; dy <= 1; ++dy) {
        for (int dx = -1; dx <= 1; ++dx) {
            int ix = min(max(x+dx,0), width-1);
            int iy = min(max(y+dy,0), height-1);
            sum += input[iy*width + ix] * kernel[dy+1][dx+1];
        }
    }
    // Offset and clamp
    int result = sum + 128;
    result = min(max(result, 0), 255);
    output[y*width + x] = static_cast<uchar>(result);
}

#define CHECK_CUDA(call) \
    { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            std::cerr << "CUDA error: " << cudaGetErrorString(err) << " at " << __FILE__ << ":" << __LINE__ << std::endl; \
            exit(1); \
        } \
    }

int main() {
    std::string input_folder = "input_images";
    std::string output_folder = "output_images";
    auto paths = get_image_paths(input_folder);

    float gaussKernel[9] = {1.f/16, 2.f/16, 1.f/16, 2.f/16, 4.f/16, 2.f/16, 1.f/16, 2.f/16, 1.f/16};

    for (const auto& path : paths) {
        std::cout << "Processing: " << path << std::endl;
        cv::TickMeter total_cpu_timer;
        total_cpu_timer.start();

        cv::Mat img = cv::imread(path, cv::IMREAD_GRAYSCALE);
        if (img.empty() || img.type() != CV_8U || img.channels() != 1) {
            std::cerr << "Invalid image: " << path << std::endl;
            continue;
        }
        int width = img.cols, height = img.rows;
        size_t size = width * height * sizeof(uchar);

        // ----------- CPU Processing -----------
        cv::TickMeter timer_cpu_gauss, timer_cpu_laplace;
        cv::Mat cpu_gauss, cpu_laplace;
        timer_cpu_gauss.start();
        cv::GaussianBlur(img, cpu_gauss, cv::Size(3, 3), 1, 1, cv::BORDER_DEFAULT);
        timer_cpu_gauss.stop();

        timer_cpu_laplace.start();
        cv::Laplacian(cpu_gauss, cpu_laplace, CV_8U, 3, 1, 128); // offset=128
        timer_cpu_laplace.stop();

        std::string cpu_output = output_folder + "/" + std::filesystem::path(path).stem().string() + "_cpu.bmp";
        if (!cv::imwrite(cpu_output, cpu_laplace)) {
            std::cerr << "Failed to save CPU image: " << cpu_output << std::endl;
        }
        total_cpu_timer.stop();

        // ----------- GPU Processing -----------
        cv::TickMeter timer_gpu_total, timer_gpu_gauss, timer_gpu_laplace;
        timer_gpu_total.start();
        uchar *d_input, *d_gauss, *d_laplace;
        float* d_gkernel;
        CHECK_CUDA(cudaMallocManaged(&d_input, size));
        CHECK_CUDA(cudaMallocManaged(&d_gauss, size));
        CHECK_CUDA(cudaMallocManaged(&d_laplace, size));
        CHECK_CUDA(cudaMallocManaged(&d_gkernel, 9 * sizeof(float)));

        CHECK_CUDA(cudaMemcpy(d_input, img.data, size, cudaMemcpyHostToDevice));
        CHECK_CUDA(cudaMemcpy(d_gkernel, gaussKernel, 9 * sizeof(float), cudaMemcpyHostToDevice));

        dim3 block(16,16), grid((width+15)/16, (height+15)/16);

        timer_gpu_gauss.start();
        gaussian_filter_kernel<<<grid, block>>>(d_input, d_gauss, width, height, d_gkernel);
        timer_gpu_gauss.stop();

        CHECK_CUDA(cudaDeviceSynchronize());
        CHECK_CUDA(cudaGetLastError());

        timer_gpu_laplace.start();
        laplacian_filter_kernel<<<grid, block>>>(d_gauss, d_laplace, width, height);
        timer_gpu_laplace.stop();
        CHECK_CUDA(cudaDeviceSynchronize());
        CHECK_CUDA(cudaGetLastError());

        cv::Mat out(height, width, CV_8U);
        CHECK_CUDA(cudaMemcpy(out.data, d_laplace, size, cudaMemcpyDeviceToHost));

        std::string gpu_output = output_folder + "/" + std::filesystem::path(path).stem().string() + "_gpu.bmp";
        if (!cv::imwrite(gpu_output, out)) {
            std::cerr << "Failed to save GPU image: " << gpu_output << std::endl;
        }

        CHECK_CUDA(cudaFree(d_input));
        CHECK_CUDA(cudaFree(d_laplace));
        CHECK_CUDA(cudaFree(d_gkernel));
        timer_gpu_total.stop();

        //total_timer.stop();
        // ------- Print stage timings --------
        std::cout << "CPU GaussianBlur:    " << timer_cpu_gauss.getTimeMilli()    << " ms" << std::endl;
        std::cout << "CPU Laplacian:       " << timer_cpu_laplace.getTimeMilli()  << " ms" << std::endl;
        std::cout << "Total image time:    " << total_cpu_timer.getTimeMilli()    << " ms" << std::endl << std::endl;

        std::cout << "GPU Gaussian kernel: " << timer_gpu_gauss.getTimeMilli()    << " ms" << std::endl;
        std::cout << "GPU Laplace kernel: " << timer_gpu_laplace.getTimeMilli() << " ms" << std::endl;
        std::cout << "GPU Total:           " << timer_gpu_total.getTimeMilli()    << " ms" << std::endl;
        std::cout << "\n";
    }
    return 0;
}


Writing cpu_vs_gpu_no_warmup.cu


In [18]:
%%shell
nvcc cpu_vs_gpu_no_warmup.cu -o cpu_vs_gpu_no_warmup $(pkg-config --cflags --libs opencv4) -arch=sm_75

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [19]:
%%shell
ls -l cpu_vs_gpu_no_warmup

-rwxr-xr-x 1 root root 1070512 Aug 15 14:58 cpu_vs_gpu_no_warmup


In [20]:
%%shell
./cpu_vs_gpu_no_warmup

Processing: input_images/greenland_grid_velo.bmp
CPU GaussianBlur:    3.45945 ms
CPU Laplacian:       1.71704 ms
Total image time:    14.4 ms

GPU Gaussian kernel: 0.120837 ms
GPU Laplace kernel: 0.012496 ms
GPU Total:           116.259 ms

Processing: input_images/lena.bmp
CPU GaussianBlur:    0.205313 ms
CPU Laplacian:       0.266634 ms
Total image time:    1.40993 ms

GPU Gaussian kernel: 0.007113 ms
GPU Laplace kernel: 0.004496 ms
GPU Total:           0.836995 ms

